### Step 1: Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd

url = "https://www.naukri.com/data-science-jobs-in-mumbai?k=data%20science&l=mumbai"

### Step 2: Getting The Page Links

In [2]:
#Setting the driver has a function
Page_Links = []
for index,links in enumerate(url):
    if index == 2:
        break
    else:
        def chromedriver(x):
            driver = webdriver.Chrome("D:\\ChromeD\\chromedriver.exe")
            driver.get(x)
            time.sleep(2)
            soup = bs(driver.page_source,"html5lib")
            driver.close()
            return soup
        page = chromedriver(url)
        next_page = page.find(class_ = "pagination mt-64 mb-60")
        page_link = next_page.find(class_ = "fright fs14 btn-secondary br2")["href"]
        base_path = f"https://www.naukri.com{page_link}"
        url = base_path
        Page_Links.append(url)

### Step 3: Scraping The Page

In [3]:
JobDesc = []
for urls in Page_Links:
    page_soup = chromedriver(urls)
    get_list = page_soup.find(class_ = "list")
    
    for index,post in enumerate(get_list.find_all(class_ = "jobTuple bgWhite br4 mb-8")):
        #getting the title
        try:
            get_title = post.find(class_ = "title fw500 ellipsis").get_text()
        except:
            get_title = " "

        #getting the company name
        try:
            get_company = post.find('a',class_='subTitle ellipsis fleft').get_text()
        except:
            get_company = " "

        #getting company rating
        try:
            get_rating = post.find('span',class_='starRating fleft dot').get_text()
        except:
            get_rating = " "

        #getting company review count
        try:
            get_review = post.find('a',class_='reviewsCount ml-5 fleft blue-text').get_text()
        except:
            get_review = " " 

        #getting the experience needed
        try:
            experience = post.find('li',class_='fleft grey-text br2 placeHolderLi experience')
            get_experience = experience.find('span',class_='ellipsis fleft fs12 lh16').get_text()
        except:
            get_experience = " "

        #getting the salary details 
        try:
            get_salary = post.find(class_ = "fleft grey-text br2 placeHolderLi salary").get_text()
        except:
            get_salary = " "

        #getting the location details
        try:
            get_location = post.find(class_ = "fleft grey-text br2 placeHolderLi location").get_text()
        except:
            get_location = " "

        #getting skills required
        try:
            get_skills = post.find(class_ = "tags has-description" ).get_text(",", strip = True)
        except:
            get_skills = " "

        #getting the post link
        try:
            get_link = post.find(class_ = "title fw500 ellipsis").get("href")
        except:
            get_link = " "

        #Storing the data in a dictionary
        Other_details = {"Company": get_company,
                "Rating" : get_rating,
                "Review Count" : get_review, 
                "Title" : get_title,
                "Experience" : get_experience,
                "Salary" : get_salary,
                "Location" : get_location,
                "Skills" : get_skills,
                "Link" : get_link,
                }    


        post_link = post.find(class_ = "title fw500 ellipsis")["href"]
        post_soup = chromedriver(post_link)

        try:
            content_list = post_soup.find(class_ = "jdContainer job-desc")
            for job_desc in content_list.find(class_ = "getJobDescriptionOtherDetails JD av_textblock_section jDisc mt25"):
                get_lab = job_desc.find("em").get_text().replace(":", "")
                get_span = job_desc.find("span").get_text()
                Other_details[get_lab] = get_span
            JobDesc.append(Other_details)
        except:
            content_list = post_soup.find(class_ = "job-desc")
            for job_desc in content_list.find_all(class_ = "details"):
                get_lab = job_desc.find("label").get_text().replace(":", "")
                get_sp = job_desc.find("span").get_text()
                Other_details[get_lab] = get_sp
            JobDesc.append(Other_details)

### Step 4: Converting List to DataFrame

In [4]:
df = pd.DataFrame(JobDesc)
df.head()

,Company,Rating,Review Count,Title,Experience,Salary,Location,Skills,Link,Role,Industry,Functional Area,Role Category,Employment Type,Industry Type,UG,PG,Doctorate
0,Capgemini Technology Services India Limited,3.6,(12552 Reviews),Data Science,6-9 Yrs,Not Disclosed by Recruiter,Mumbai,"Machine Learning,communication,Python,analysis...",https://www.naukri.com/job-listings-data-scien...,Database Architect/Designer,IT Services & Consulting,"IT Software - eCommerce, Internet Technologies",Programming & Design,"Full Time, Permanent",NaN,NaN,NaN,NaN
1,Data is Good,,,Data Science Content Developer,0-1 Yrs,"1,75,000 - 2,50,000 PA.","Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...","Data Science,Python Dash,Excel,Content Develop...",https://www.naukri.com/job-listings-data-scien...,"Data Analyst,",NaN,"Analytics & Business Intelligence,",Analytics & BI,"Full Time, Permanent","Education / Training,",Any Graduate,NaN,NaN
2,Amploy.io,,,Data Science Intern (remote),0-5 Yrs,"13,00,000 - 18,00,000 PA.","Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...","Data Science,Application Programming,Software ...",https://www.naukri.com/job-listings-data-scien...,"Software Developer,",NaN,"IT Software - Application Programming, Mainten...",Programming & Design,"Full Time, Permanent","IT Services & Consulting,",Graduation Not Required,NaN,NaN
3,Reliance Jio Infocomm Ltd.,4.0,(10076 Reviews),Data Science Engineer,3-5 Yrs,Not disclosed,Mumbai,"Data analysis,Statistical modeling,data scienc...",https://www.naukri.com/job-listings-data-scien...,"Database Architect/Designer,",NaN,"IT Software - Application Programming, Mainten...",Programming & Design,"Full Time, Permanent","Telecom / ISP,",Any Graduate,M.Tech in Electronics/Telecommunication,NaN
4,Sankhkyiki Robotics,,,Data Science Intern,0-1 Yrs,Not disclosed,Mumbai,"IT Skills,Data Science,Machine Learning,Artifi...",https://www.naukri.com/job-listings-data-scien...,"Software Developer,",NaN,"IT Software - Application Programming, Mainten...",Programming & Design,"Full Time, Permanent","IT Services & Consulting,",B.Tech/B.E. in Computers,Post Graduation Not Required,NaN


### Step 5: Saving the DataFrame to Csv

In [5]:
df.to_csv("Mumbai Ds Job DataSet.csv")